# Seyed Mohammad Amin Atyabi - 830402014

# 1 - Maximum Likelihood Estimation

**1**

<img src="res/1-1.jpg" width="1280">

**2**

<img src="res/1-2.jpg" width="1280">

**3**

<img src="res/1-3.jpg" width="1280">

# 2 - EM Algorithm

# 3 - Method of Moments

**1**

<img src="res/3-1.jpg" width="1280">

**2**

<img src="res/3-2.jpg" width="1280">

# 4 - Speech Recognition Model Using Multi Layer Perceptron

در این بخش با استفاده از کتابخانه پای تورچ و پلتفرم گوگل کولب به دسته بندی داده های صوتی خواهیم پرداخت

برخی از چالش های موجود در انجام این بخش از تمرین عبارتند از کم بودن حافظه رم کولب و نیاز به پردازنده گرافیکی هستند که در ادامه راه حل آن ها توضیح داده خواهد شد

In [1]:
!pip install torchsummaryX wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


با تنظیم گوگل کولب بر استفاده از پردازنده گرافیکی ، در صورت وجود داشتن رانر این پردازنده به ما اختصاص میابد

پردازنده CPU با توجه به کند بودن باعث صرف زمان بسیار زیاد در  آموزش و ارزیابی مدل میشد که قابل استفاده نبود اما پردازنده گرافیکی به مراتب سریعتر است

مشکل استفاده از این نوع پردازنده دردسترس نبودن همیشگی آن و در دسترس بودن آن تنها در برخی ساعت پایانی شب بود که انجام این تمرین را خود به چالشی از نظر زمانی کرده بود . همچنین لازم بود با صفحه وب تعامل داشت در غیر این صورت نشست به دلیل عدم فعالیت بسته میشد

In [2]:
import torch
import numpy as np
from torchsummaryX import summary
import gc
from tqdm.auto import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
!gdown 1gNVILsmbYZbYJQk2-NN2q7VG2RKJp-7x

Downloading...
From: https://drive.google.com/uc?id=1gNVILsmbYZbYJQk2-NN2q7VG2RKJp-7x
To: /content/Data.zip
100% 4.28G/4.28G [00:49<00:00, 87.0MB/s]


In [ ]:
!unzip Data.zip -d dataset
!rm Data.zip

In [5]:
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

در این بخش به دلیل کم بودن حافظه رم کولب و مواجه شدن با مشکلاتی ، صرفا یک سوم داده ها خوانده میشوند

In [6]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes=PHONEMES, context=0, partition="train-clean-100"):

        self.context = context
        self.phonemes = phonemes

        self.mfcc_dir = root + "mfcc/"
        self.transcript_dir = root + "transcript/"

        mfcc_names = os.listdir(self.mfcc_dir)
        transcript_names = os.listdir(self.transcript_dir)

        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        item = 0

        for m, t in zip(mfcc_names, transcript_names):
            item += 1
            if (item % 3) != 0: # Only load 33 percent of data due to colab memory limitaions
              continue
            mfcc = np.load(self.mfcc_dir + m)
            transcript = np.load(self.transcript_dir + t)
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        self.mfccs = np.concatenate(self.mfccs, axis=0)

        self.transcripts = np.concatenate(self.transcripts, axis=0)
        self.length = len(self.mfccs)

        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0)

        self.transcripts = [PHONEMES.index(transcript) for transcript in self.transcripts]

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        ind = ind + self.context
        frames = self.mfccs[ind - self.context: ind + self.context + 1]
        frames = frames.flatten()

        frames = torch.FloatTensor(frames)
        phonemes = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [7]:
class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, phonemes=PHONEMES, context=0, partition="train-clean-100"):
        self.context = context
        self.phonemes = phonemes

        self.mfcc_dir = root + "mfcc/"
        mfcc_names = os.listdir(self.mfcc_dir)

        self.mfccs = []

        for m in mfcc_names:
            mfcc = np.load(self.mfcc_dir + m)
            self.mfccs.append(mfcc)
        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.length = len(self.mfccs)
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values=0)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        ind = ind + self.context
        frames = self.mfccs[ind - self.context: ind + self.context + 1].flatten()
        frames = torch.FloatTensor(frames)

        return frames


In [8]:
train_loader = None
val_loader = None
test_loader = None
criterion = None
optimizer = None
config = None
model = None
train_data = None
val_data = None
test_data = None

کد های این بخش سعی شده تا به گونه ای طی چند تابع اصلاح شوند تا در ادامه به سادگی بتوان با پیکربندی و شبکه جدید آن ها را اجرا کرد

پاسخ سوال چرایی در هم ریختن داده ها : داده آموزش در هم ریخته میشود تا از  سو گیری و بیش برازش جلوگیری کنیم اما داده آزمون و ارزیابی برای ثابت بودن معیار آزمون و ارزیابی در هم ریخته نمیشوند

In [9]:
def load_data():

  global train_data
  global val_data
  global test_data

  train_data = AudioDataset("dataset/11-785-f23-hw1p2/train-clean-100/",context=config['context'])
  val_data = AudioDataset("dataset/11-785-f23-hw1p2/dev-clean/",context=config['context'])
  test_data = AudioTestDataset("dataset/11-785-f23-hw1p2/test-clean/", context=config['context'])


def run_network():

  global train_loader
  global val_loader
  global test_loader

  train_loader = torch.utils.data.DataLoader(
      dataset=train_data,
      num_workers=2,
      batch_size=config['batch_size'],
      pin_memory=True,
      shuffle=True
  )

  val_loader = torch.utils.data.DataLoader(
      dataset=val_data,
      num_workers=2,
      batch_size=config['batch_size'],
      pin_memory=True,
      shuffle=False
  )

  test_loader = torch.utils.data.DataLoader(
      dataset=test_data,
      num_workers=2,
      batch_size=config['batch_size'],
      pin_memory=True,
      shuffle=False
  )

  print("Batch size     : ", config['batch_size'])
  print("Context        : ", config['context'])
  print("Input size     : ", (2 * config['context'] + 1) * 28)
  print("Output symbols : ", len(PHONEMES))

  print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
  print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
  print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

  for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

  global criterion
  global optimizer
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr'])

  torch.cuda.empty_cache()
  gc.collect()

def train(model, dataloader, optimizer, criterion):
    model.train()
    tloss, tacc = 0, 0
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):
        optimizer.zero_grad()

        frames = frames.to(device)
        phonemes = phonemes.to(device)

        logits = model(frames)

        loss = criterion(logits, phonemes)

        loss.backward()

        optimizer.step()

        tloss += loss.item()
        tacc += torch.sum(torch.argmax(logits, dim=1) == phonemes).item() / logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc * 100 / (i + 1))))
        batch_bar.update()

        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss /= len(train_loader)
    tacc /= len(train_loader)

    return tloss, tacc

def eval(model, dataloader):
    model.eval()
    vloss, vacc = 0, 0
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):
        frames = frames.to(device)
        phonemes = phonemes.to(device)

        with torch.inference_mode():
            logits = model(frames)
            loss = criterion(logits, phonemes)

        vloss += loss.item()
        vacc += torch.sum(torch.argmax(logits, dim=1) == phonemes).item() / logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc * 100 / (i + 1))))
        batch_bar.update()

        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss /= len(val_loader)
    vacc /= len(val_loader)

    return vloss, vacc

def run_train_eval():
  torch.cuda.empty_cache()
  gc.collect()

  for epoch in range(config['epochs']):
      print("\nEpoch {}/{}".format(epoch + 1, config['epochs']))

      curr_lr = float(optimizer.param_groups[0]['lr'])
      train_loss, train_acc = train(model, train_loader, optimizer, criterion)
      val_loss, val_acc = eval(model, val_loader)

      print(
          "\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc * 100, train_loss, curr_lr))
      print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc * 100, val_loss))

اندازه داده ورودی : دو برابر طول بردار زمینه بعلاوه اندیس مرکز ، برای مثال به ازای بردار زمینه با اندازه 3 ، 3 اندیس بعد و قبل از اندیس مرکزی نیاز است . از آن جایی که برای هر بردار ، یک بردار 28 تایی وجود دارد ، اندازه  داده ورودی در 28 ضرب میشود

در ادامه شبکه های مختلفی آزموده خواهند شد

شبکه زیر شبکه بسیار ساده از با حدود 65 هزار پارامتر است . در آموزش این مدل هرچه به ایپاک های جلوتر میرویم دقت داده آموزش بیشتر میشود ولی از داده آزمون کم میشود که این نشان از بیش برازش مدل بر داده آموزش دارد

In [ ]:
config = {
    'epochs'        : 5,
    'batch_size'    : 1024,
    'context'       : 1,
    'init_lr'       : 1e-3,
    'architecture'  : 'very-low-cutoff'
}

class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, output_size)
        )

    def forward(self, x):
        out = self.model(x)
        return out

load_data()
INPUT_SIZE  = (2*config['context'] + 1) * 28
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
frames, _ = train_data.__getitem__(0)
summary(model, frames.to(device))

run_network()
run_train_eval()

                 Kernel Shape Output Shape   Params Mult-Adds
Layer                                                        
0_model.Linear_0    [84, 512]        [512]   43.52k   43.008k
1_model.ReLU_1              -        [512]        -         -
2_model.Linear_2    [512, 42]         [42]  21.546k   21.504k
--------------------------------------------------------------
                       Totals
Total params          65.066k
Trainable params      65.066k
Non-trainable params      0.0
Mult-Adds             64.512k
Batch size     :  1024
Context        :  1
Input size     :  84
Output symbols :  42
Train dataset samples = 7254393, batches = 7085
Validation dataset samples = 393325, batches = 385
Test dataset samples = 1934138, batches = 1889


/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


torch.Size([1024, 84]) torch.Size([1024])

Epoch 1/5


Train:   0%|          | 0/7085 [00:00<?, ?it/s]

Val:   0%|          | 0/385 [00:00<?, ?it/s]

...

Val:   0%|          | 0/385 [00:00<?, ?it/s]

	Train Acc 17.2083%	Train Loss 3.3014	 Learning Rate 0.0010000
	Val Acc 16.4002%	Val Loss 3.3228

Epoch 5/5


Train:   0%|          | 0/7085 [00:00<?, ?it/s]

Val:   0%|          | 0/385 [00:00<?, ?it/s]

	Train Acc 17.2138%	Train Loss 3.3010	 Learning Rate 0.0010000
	Val Acc 16.3994%	Val Loss 3.3236


در این بخش سعی شد با اضافه کردن لایه های میانی و dropout از برازش بیش از اندازه جلوگیری شد که نتایج آن در داده ارزیابی قابل مشاهده است

In [ ]:
config = {
    'epochs'        : 10,
    'batch_size'    : 1024,
    'context'       : 1,
    'init_lr'       : 1e-3,
    'architecture'  : 'very-low-cutoff'
}

class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Sigmoid(),
            torch.nn.Linear(512, output_size)
        )

    def forward(self, x):
        out = self.model(x)
        return out

load_data()
INPUT_SIZE  = (2*config['context'] + 1) * 28
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
frames, _ = train_data.__getitem__(0)
summary(model, frames.to(device))

run_network()
run_train_eval()

                  Kernel Shape Output Shape   Params Mult-Adds
Layer                                                         
0_model.Linear_0     [84, 512]        [512]   43.52k   43.008k
1_model.ReLU_1               -        [512]        -         -
2_model.Dropout_2            -        [512]        -         -
3_model.Sigmoid_3            -        [512]        -         -
4_model.Linear_4     [512, 42]         [42]  21.546k   21.504k
---------------------------------------------------------------
                       Totals
Total params          65.066k
Trainable params      65.066k
Non-trainable params      0.0
Mult-Adds             64.512k
Batch size     :  1024
Context        :  1
Input size     :  84
Output symbols :  42
Train dataset samples = 7254393, batches = 7085
Validation dataset samples = 393325, batches = 385
Test dataset samples = 1934138, batches = 1889


/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


torch.Size([1024, 84]) torch.Size([1024])

Epoch 1/10


Train:   0%|          | 0/7085 [00:00<?, ?it/s]

Val:   0%|          | 0/385 [00:00<?, ?it/s]

...

Val:   0%|          | 0/385 [00:00<?, ?it/s]

	Train Acc 17.1871%	Train Loss 3.3103	 Learning Rate 0.0010000
	Val Acc 16.4045%	Val Loss 3.3359

Epoch 10/10


Train:   0%|          | 0/7085 [00:00<?, ?it/s]

Val:   0%|          | 0/385 [00:00<?, ?it/s]

	Train Acc 17.1870%	Train Loss 3.3101	 Learning Rate 0.0010000
	Val Acc 16.4045%	Val Loss 3.3337


در این بخش سعی شد با پیشرفته تر کردن مدل بخش قبلی به دقت بالاتری دست پیدا کنیم و تعداد پارامتر هارا تا مرز تعداد پارامتر ها زیاد کنیم اما متاسفانه با مشاهده نتایج ضعیف آن از  اجرای بیشتر آن جلوگیری شد

In [10]:
config = {
    'epochs'        : 20,
    'batch_size'    : 1024,
    'context'       : 32,
    'init_lr'       : 1e-3,
    'architecture'  : 'very-low-cutoff'
}

class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        self.model = torch.nn.Sequential(

            torch.nn.Linear(input_size, 1024),
            torch.nn.Dropout(p=0.1),

            torch.nn.Linear(1024, 2048),
            torch.nn.Dropout(p=0.1),

            torch.nn.Linear(2048, 4096),
            torch.nn.Dropout(p=0.1),

            torch.nn.Linear(4096, 2048),
            torch.nn.Dropout(p=0.1),

            torch.nn.Linear(2048, 1024),
            torch.nn.Dropout(p=0.1),

            torch.nn.Linear(1024, 512),
            torch.nn.Dropout(p=0.1),

            torch.nn.Linear(512, output_size)
        )

    def forward(self, x):
        out = self.model(x)
        return out

load_data()
INPUT_SIZE  = (2*config['context'] + 1) * 28
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
frames, _ = train_data.__getitem__(0)
summary(model, frames.to(device))

run_network()
run_train_eval()

                     Kernel Shape Output Shape     Params  Mult-Adds
Layer                                                               
0_model.Linear_0     [1820, 1024]       [1024]  1.864704M   1.86368M
1_model.Dropout_1               -       [1024]          -          -
2_model.Linear_2     [1024, 2048]       [2048]    2.0992M  2.097152M
3_model.Dropout_3               -       [2048]          -          -
4_model.Linear_4     [2048, 4096]       [4096]  8.392704M  8.388608M
5_model.Dropout_5               -       [4096]          -          -
6_model.Linear_6     [4096, 2048]       [2048]  8.390656M  8.388608M
7_model.Dropout_7               -       [2048]          -          -
8_model.Linear_8     [2048, 1024]       [1024]  2.098176M  2.097152M
9_model.Dropout_9               -       [1024]          -          -
10_model.Linear_10    [1024, 512]        [512]     524.8k   524.288k
11_model.Dropout_11             -        [512]          -          -
12_model.Linear_12      [512, 42] 

/usr/local/lib/python3.10/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


torch.Size([1024, 1820]) torch.Size([1024])

Epoch 1/20


Train:   0%|          | 0/11744 [00:00<?, ?it/s]

Val:   0%|          | 0/630 [00:00<?, ?it/s]

...

Train:   0%|          | 0/11744 [00:00<?, ?it/s]

Val:   0%|          | 0/630 [00:00<?, ?it/s]

	Train Acc 5.4117%	Train Loss 83173.6764	 Learning Rate 0.0010000
	Val Acc 15.3567%	Val Loss 610.3696

Epoch 8/20


Train:   0%|          | 0/11744 [00:00<?, ?it/s]

KeyboardInterrupt: 

مدل زیر با ایده گرفتن در پاسخ های چت بات بینگ تولید شده است . خلاصه مکالمه صورت گرفته بدین شرح بود :

I am doing frame level speech recognition.
Can you give me examples of pytorch model for classification?

پاسخ بینگ برخی توضیحات در خصوص  پای تورچ بود

Would you please show me the code for MFCC?

پاسخ بینگ نحوه پلات کردن داده صوتی بود

Show me the code using pytorch for classification MFCC

پاسخ بینگ مدلی شبیه به اولین مدل نوشته شده توسط ما بود

very well
make the nn model more advanced

پاسخ بینگ در این بخش شامل Linear و BatchNorm1d و GELU و Dropout بود که با کمی جست و جو قرار دادن آن ها پشت سر هم این مدل بوجود آمد

همانطور که در نتایج این مدل نیز مشخص است بالاترین دقت در داده آموزش توسط این مدل بدست آمد اما هرچه جلوتر میرفت به دلیل برازش بیش از حد بر روی داده آموزش ، دقت مدل بر روی داده ارزیابی کمتر و کمتر میشد

In [13]:
config = {
    'epochs'        : 20,
    'batch_size'    : 1024,
    'context'       : 32,
    'init_lr'       : 1e-3,
    'architecture'  : 'very-low-cutoff'
}

class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):
        super(Network, self).__init__()
        self.model = torch.nn.Sequential(

            torch.nn.Linear(input_size, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.2),

            torch.nn.Linear(1024, 1024),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.2),

            torch.nn.Linear(1024, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.2),

            torch.nn.Linear(1024, 1024),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.2),

            torch.nn.Linear(1024, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.2),

            torch.nn.Linear(1024, output_size)
        )

    def forward(self, x):
        out = self.model(x)
        return out

load_data()
INPUT_SIZE  = (2*config['context'] + 1) * 28
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
frames, _ = train_data.__getitem__(0)
# summary(model, frames.to(device))

run_network()
run_train_eval()

Batch size     :  1024
Context        :  32
Input size     :  1820
Output symbols :  42
Train dataset samples = 12025072, batches = 11744
Validation dataset samples = 644779, batches = 630
Test dataset samples = 1934138, batches = 1889
torch.Size([1024, 1820]) torch.Size([1024])

Epoch 1/20


Train:   0%|          | 0/11744 [00:00<?, ?it/s]

Val:   0%|          | 0/630 [00:00<?, ?it/s]

	Train Acc 17.1749%	Train Loss 3.3066	 Learning Rate 0.0010000
	Val Acc 16.5585%	Val Loss 3.3184

Epoch 2/20


Train:   0%|          | 0/11744 [00:00<?, ?it/s]

...

Val:   0%|          | 0/630 [00:00<?, ?it/s]

	Train Acc 20.7213%	Train Loss 3.0481	 Learning Rate 0.0010000
	Val Acc 11.5620%	Val Loss 3.4938

Epoch 20/20


Train:   0%|          | 0/11744 [00:00<?, ?it/s]

Val:   0%|          | 0/630 [00:00<?, ?it/s]

	Train Acc 20.8356%	Train Loss 3.0413	 Learning Rate 0.0010000
	Val Acc 11.2171%	Val Loss 3.5046


# 5 - Clustering

**1**

<img src="res/5-1.jpg" width="1280">

**2**

<img src="res/5-2.jpg" width="1280">

# 6 - K-Means Clustering

**1**

<img src="res/6-1.jpg" width="1280">

**2**

<img src="res/6-2.jpg" width="1280">